# Vaje 6: Induktivni podatkovni tipi — AVL drevesa

Implementirali bomo [AVL drevesa](https://en.wikipedia.org/wiki/AVL_tree) v jeziku OCaml. AVL drevo je dvojiško iskalno drevo, v katerem se za vsako vozlišče višini levega in desnega poddrevesa razlikujeta za največ 1.

## Podatkovni tip AVL drevo

V splošnem bi želeli v iskalnih drevesih hraniti poljubne podatke, ki jih lahko primerjamo po velikosti. Tu bomo nalogo poenostavili tako, da bomo v drevesih hranili le cela števila (`int`).

AVL drevo je:

* bodisi prazno
* bodisi vozlišče, sestavljeno iz:
    * vsebine (števila),
    * višine drevesa in
    * levega ter desnega AVL poddrevesa.

**Naloga:** definirajte podatkovni tip `avltree`, ki opisuje AVL drevesa.

In [2]:
type height = int
type value = int

type avltree =
  | Empty
  | Node of value * height * avltree * avltree (* Node constructor takes in 4 arguments, value, height, avltree, and avltree*)

type height = int


type value = int


type avltree = Empty | Node of value * height * avltree * avltree


Spodnja funkcja `showTreeHorizontally : avltree -> unit` grafično prikaže vhodno drevo `t` po nivojih.

In [4]:
let showTreeHorizontally t = 
    let rec strign_of_avltree_level lvl = function 
        | Empty -> if lvl = 0 then "nil" else "   " 
        | Node (n, h, l, r) ->
              let make_space = String.map (fun _ -> ' ') in
              let sn = string_of_int n in
              let sl = strign_of_avltree_level lvl l in
              let sr = strign_of_avltree_level lvl r in
                  if h = lvl
                  then make_space sl ^ sn ^ make_space sr
                  else sl ^ make_space sn ^ sr
    in
    let rec print_levels lvl =
        if lvl >= 0
        then (print_string (string_of_int lvl ^ ": " ^ strign_of_avltree_level lvl t ^ "\n");
            print_levels (lvl-1))
        else ()
    in
    let height = function
        | Node (_, y, _, _) -> y
        | Empty -> 0
    in
    print_levels (height t) ; flush stdout

val showTreeHorizontally : avltree -> unit = <fun>


In [5]:
let t =
    Node (5, 3,
        Node (3, 2,
            Node(1, 1, Empty, Empty),
            Node(4, 1, Empty, Empty)
        ),
        Node (8, 1, Empty, Empty)
    ) ;;

showTreeHorizontally t ;;

val t : avltree =
  Node (5, 3,
   Node (3, 2, Node (1, 1, Empty, Empty), Node (4, 1, Empty, Empty)),
   Node (8, 1, Empty, Empty))


3:                5       
2:        3               
1:    1       4       8   
0: nil nil nil nil nil nil


- : unit = ()


In [9]:
(* Function that returns the height of the tree *)
let height = function
    | Empty -> 0
    | Node (_, h, _, _) -> h ;;

height Empty ;;
height t ;;

val height : avltree -> height = <fun>


- : height = 0


- : height = 3


**Naloga:** definirajte "pametna" konstruktorja `leaf: int -> avltree` in `node: int * avltree * avltree -> avltree`, ki sama poskrbita za višino. Prav vam bo prišla funkcija `max: int -> int -> int`, ki vrne večjega od dveh števil. Primer uporabe:

    # let six = leaf 6 ;;
    val six : avltree = Node (6, 1, Empty, Empty)
    # let seven = leaf 7 ;;
    val seven : avltree = Node (7, 1, Empty, Empty)
    # let answer = node (42, six, seven) ;;
    val answer : avltree =
       Node (42, 2, Node (6, 1, Empty, Empty), Node (7, 1, Empty, Empty))

In [12]:
let leaf x = Node (x, 1, Empty, Empty) ;;
let six = leaf 6;;
let seven = leaf 7;;

let node (v, x, y) = Node(v, 1 + max (height x) (height y), x, y) ;;
let answer = node(42, six, seven) ;;

val leaf : value -> avltree = <fun>


val six : avltree = Node (6, 1, Empty, Empty)


val seven : avltree = Node (7, 1, Empty, Empty)


val node : value * avltree * avltree -> avltree = <fun>


val answer : avltree =
  Node (42, 2, Node (6, 1, Empty, Empty), Node (7, 1, Empty, Empty))


**Naloga:** s pametnimi konstruktorji definirajte AVL drevo, enako drevesu `t` iz prejšnje naloge.

In [15]:
let t2 = node(5, node(3, leaf 1, leaf 4), leaf 8) ;;
showTreeHorizontally t2 ;;

val t2 : avltree =
  Node (5, 3,
   Node (3, 2, Node (1, 1, Empty, Empty), Node (4, 1, Empty, Empty)),
   Node (8, 1, Empty, Empty))


3:                5       
2:        3               
1:    1       4       8   
0: nil nil nil nil nil nil


- : unit = ()


## Drevo ⇒ seznam

**Naloga:** definirajte funkcijo `toList: avltree -> int list`, ki elemente drevesa vrne kot urejen seznam števil. Za združevanje seznamov ima OCaml operator `@`.

In [19]:
(* rešitev *)
let rec toList = function
    | Empty -> []
    | Node (x, _, l, r) -> toList l @ [x] @ toList r ;;

(* ena izmed neučinkovitih implementacij stikanja seznamov *)
let rec join_list xs ys = match xs with
    | [] -> ys
    | glava :: rep -> glava :: join_list rep ys ;;

toList t2 ;;

val toList : avltree -> value list = <fun>


val join_list : 'a list -> 'a list -> 'a list = <fun>


- : value list = [1; 3; 4; 5; 8]


## Iskanje

Algoritem, ki ugotovi, ali je dani `x` v drevesu `t`:

* če je `t` prazno drevo, se `x` ne pojavi;
* če je `t` vozlišče z vsebino `y` in poddrevesoma `l` ter `r`:
    * če je `x = y`, se `x` pojavi;
    * če je `x < y`, iščemo v poddrevesu `l`;
    * če je `x > y`, iščemo v poddrevesu `r`.

Iskanje bomo implementirali s funkcijo `search`, ki naj deluje tako:

    # search 5 t ;;
    - : bool = true

**Naloga** zapišite *tip* funkcije `search`, ki ugotovi, ali drevo `t` vsebuje vrednost `x`.

In [20]:
let rec search x = function
    | Empty -> false
    | Node (y, _, l, r) ->
        if x = y then true
        else if x < y then search x l
        else search x r ;;

search 5 t2 ;;

val search : value -> avltree -> bool = <fun>


- : bool = true


**Naloga** definirajte funkcijo `search`. Za primerjanje dveh števil uporabite funkcijo `cmp` s predavanj, ki vrne vrednost tipa `order`:   

In [22]:
type order = Less | Equal | Greater

let cmp x y =
    match compare x y with
    | 0 -> Equal
    | r when r < 0 -> Less
    | _ -> Greater

type order = Less | Equal | Greater


val cmp : 'a -> 'a -> order = <fun>


In [23]:
let rec searcg x = function
    | Empty -> false
    | Node (y, _, l, r) ->
        match cmp x y with
        | Equal -> true
        | Less -> search x l
        | Greater -> search x r ;;


(* počasna rešitev *)
let search_slow e t = List.mem e (toList t)

let test1 = search 1 t ;;
let test2 = search 42 t ;;

val searcg : value -> avltree -> bool = <fun>


val search_slow : value -> avltree -> bool = <fun>


val test1 : bool = true


val test2 : bool = false


## Vrtenje in uravnoteženje

Pri vstavljanju ali brisanju elementov se lahko AVL drevo "pokvari": višina levega in desnega poddrevesa nekega poddrevesa se razlikujeta za več kot 1. To popravimo z vrtenjem (rotacijo) drevesa okrog korena.

**Naloga:** definirajte funkcijo `imbalance: avltree -> int`, ki vrne stopnjo neuravnoteženosti drevesa, tj. razliko med višinama levega in desnega poddrevesa.

    # imbalance t ;;
    - : int = 1
    # imbalance Empty ;;
    - : int = 0       

In [ ]:
let imbalance = function
    | Empty -> 0
    | Node (_, _, l, r) -> height l - height r ;;

V AVL drevesu je lahko neuravnoteženost kateregakoli vozlišča največ 1. Bolj neuravnotežena poddrevesa lahko popravimo z vrtenjem v levo oziroma desno. Vrtenje v levo oziroma desno ponazorimo z diagramom

         x                      y
       /   \       levo       /   \
      +     y       ==>      x     +
     /a\   / \              / \   /c\
     ---  +   +     <==    +   +  ---
         /b\ /c\   desno  /a\ /b\
         --- ---          --- ---
         
**Naloga:** definirajte funkciji `rotateLeft` in `rotateRight`, ki dano drevo zavrtita na prikazan način. Če to ni mogoče (ker je drevo npr. prazno ali list), naj funkciji vrneta nespremenjeno drevo.

In [24]:
let rotateLeft = function
    | Node (x, _, a, Node (y, _, b, c)) -> node (y, node (x, a, b), c)
    | t -> t

let rotateRight = function
    | Node (y, _, Node (x, _, a, b), c) -> node (x, a, node (y, b, c))
    | t -> t

val rotateLeft : avltree -> avltree = <fun>


val rotateRight : avltree -> avltree = <fun>


Funkciji `rotateLeft` in `rotateRight` uporabimo za uravnoteženje drevesa. Ker bomo drevo uravnotežili po vsakem vstavljanju in brisanju elementa, lahko predpostavimo, da bo neuravnoteženost drevesa kvečjemu 2. Če je neuravnoteženost enaka 0, 1 ali -1, ni treba storiti ničesar. Sicer je neuravnoteženost 2 oziroma -2. V prvem primeru imamo drevo oblike

            x
          /   \
         /     \
        y       +
       / \     / \
      /   \   / h \
     / h+2 \  -----
    /       \
    ---------

Glede na višini poddreves vozlišča `y` lahko to drevo uravnotežimo z enim ali dvema vrtenjema. V prvem primeru imamo drevo oblike

             x                       y
           /   \                   /   \
          /     \                 /     \
         y       +               +        x
       /   \    /h\     ==>     / \     /   \
      +     +   ---            /h+1\   +     +
     / \   / \   c             -----  / \   /h\
    /h+1\ /h+1\*                 a   /h+1\  ---
    ----- -----                      -----   c
      a     b (lahko ima višino h)     b

ki ga popravimo z enim vrtenjem v desno, kot je prikazano zgoraj.

V drugem primeru imamo drevo oblike

            x                       x                       z
          /   \                   /   \                   /   \
         /     \                 /     \                 /     \
        y       +               z       +               y       x
      /   \    /h\    ==>     /   \     /h\    ==>     / \     / \
     +     z   ---           y     +    ---           +   +   +   +
    /h\   / \   c           / \   /h\    c           /h\ /h\ /h\ /h\
    ---  +   +             +   +  ---                --- --- --- ---
     a  /h\ /h\           /h\ /h\  b''                a   b'  b'' c
        --- ---           --- ---
         b'  b''           a   b'
     
    (b' ali b'' lahko ima višino h-1)

ki ga popravimo tako, da najprej levo poddrevo (s korenom `y`) zavrtimo v levo, nato pa celotno drevo zavrtimo v desno.

Če je neuravnoteženost drevesa enaka -2, postopamo simetrično.

**Naloga:** definirajte funkcijo `balance: avltree -> avltree`, ki uravnoteži AVL drevo na podlagi opisanih primerov.

In [ ]:
let balance = function
  | Empty -> Empty
  | Node (x, _, l, r) as t ->
      begin match imbalance t with
      | 2 ->
          begin match imbalance l with
          | -1 -> rotateRight (node (x, rotateLeft l, r))
          | _ -> rotateRight t
          end
      | -2 ->
          begin match imbalance r with
          | 1 -> rotateLeft (node (x, l, rotateRight r))
          | _ -> rotateLeft t
          end
      | _ -> t
      end

## Vstavljanje

Nov element `x` vstavimo v AVL drevo `t` po naslednjem postopku:

* če je drevo prazno, vrni list z vsebino `x`;
* če je `t` vozlišče z vsebino `y` in poddrevesoma `l` ter `r`:
    * če je `x = y`, vrni `t`;
    * če je `x < y`, vstavi `x` v `l` in rezultat uravnoteži;
    * če je `x > y`, vstavi `x` v `r` in rezultat uravnoteži.

**Naloga:** definirajte funkcijo `add: int -> avltree -> avltree`.

In [ ]:
let rec add x = function
  | Empty -> leaf x
  | Node (y, _, l, r) as t ->
      begin match cmp x y with
      | Equal -> t
      | Less -> balance (node (y, add x l, r))
      | Greater -> balance (node (y, l, add x r))
      end

## Brisanje

Element `x` iz AVL drevesa izbrišemo tako, da ga zamenjamo z njegovim naslednikom iz poddrevesa v vozlišču `x`. Pri tem bomo uporabili pomožno funkcijo `removeSuccessor`, ki vrne novo drevo in naslednika, deluje pa tako:

* če je `t`prazno drevo, sproži izjemo;
* če je `t` vozlišče z vsebino `x` in poddrevesoma `l` ter `r`:
    * če je `l` prazno drevo, vrni `(r, x)`;
    * sicer izbriši naslednika iz `l`, da dobiš `(l', y)`, nato pa sestavi in uravnoteži novo drevo s korenom `x` ter poddrevesoma `l'` in `r`.

**Naloga:** definirajte funkcijo `removeSuccessor: avltree -> avltree * int`.

In [ ]:
let 

Element `x` lahko potem izbrišemo iz AVL drevesa `t` po naslednjem postopku:

* če je `t` prazno drevo, vrni `t`;
* če je `t` vozlišče z vsebino `y` in poddrevesoma `l` in `r`:
    * če je `x < y`, izbriši `x` iz levega poddrevesa `l` in rezultat uravnoteži;
    * če je `x > y`, izbriši `x` iz desnega poddrevesa `r` in rezultat uravnoteži;
    * če je `x = y`:
        * če je desno poddrevo prazno, vrni `l`;
        * če je levo poddrevo prazno, vrni `r`;
        * sicer iz `r` izbriši naslednika, da dobiš `(r', y')`, nato pa sestavi in uravnoteži novo drevo s korenom `y'` ter poddrevesoma `l` in `r'`.
        
**Naloga:** definirajte funkcijo `remove: int -> avltree -> avltree`.

In [ ]:
let rec remove x = function
  | Empty -> Empty
  | Node (y, _, l, r) ->
      let rec removeSuccessor = function
        | Empty -> failwith "impossible"
        | Node (x, _, Empty, r) -> (r, x)
        | Node (x, _, l, r) ->
            let (l', y) = removeSuccessor l in
            (balance (node (x, l', r)), y)
      in
      match cmp x y with
      | Less -> balance (node (y, remove x l, r))
      | Greater -> balance (node (y, l, remove x r))
      | Equal ->
          begin match (l, r) with
          | (_, Empty) -> l
          | (Empty, _) -> r
          | _ -> let (r', y') = removeSuccessor r in
            balance (node (y', l, r'))
          end

In [ ]:
(* testi *)
let tr = Empty;;
let _ = showTreeHorizontally tr;;

let tr = add 1 tr;;
let test1 = tr = Node (1, 1, Empty, Empty) ;;
let _ = showTreeHorizontally tr;;


let tr = add 2 tr;;
let test2 = tr = Node (1, 2, Empty, Node (2, 1, Empty, Empty))
let _ = showTreeHorizontally tr;;

let tr = add 3 tr;;
let test3 = tr = Node (2, 2, Node (1, 1, Empty, Empty), Node (3, 1, Empty, Empty))
let _ = showTreeHorizontally tr;;

let tr = add 4 tr;;
let test4 = tr = Node (2, 3, Node (1, 1, Empty, Empty),
   Node (3, 2, Empty, Node (4, 1, Empty, Empty)))
let _ = showTreeHorizontally tr;;

let tr = add 5 tr;;
let test5 = tr = Node (2, 3, Node (1, 1, Empty, Empty),
   Node (4, 2, Node (3, 1, Empty, Empty), Node (5, 1, Empty, Empty)))
let _ = showTreeHorizontally tr;;

let tr = add 6 tr;;
let test6 = tr =  Node (4, 3,
   Node (2, 2, Node (1, 1, Empty, Empty), Node (3, 1, Empty, Empty)),
   Node (5, 2, Empty, Node (6, 1, Empty, Empty)))
let _ = showTreeHorizontally tr;;

let tr = add 7 tr;;
let test7 = tr = Node (4, 3,
   Node (2, 2, Node (1, 1, Empty, Empty), Node (3, 1, Empty, Empty)),
   Node (6, 2, Node (5, 1, Empty, Empty), Node (7, 1, Empty, Empty)))
let _ = showTreeHorizontally tr;;


let tr = remove 1 tr;;
let test_remove1 = tr = Node (4, 3, Node (2, 2, Empty, Node (3, 1, Empty, Empty)),
   Node (6, 2, Node (5, 1, Empty, Empty), Node (7, 1, Empty, Empty)))
let _ = showTreeHorizontally tr;;

let tr = remove 2 tr;;
let test_remove1 = tr = Node (4, 3, Node (3, 1, Empty, Empty),
   Node (6, 2, Node (5, 1, Empty, Empty), Node (7, 1, Empty, Empty)))
let _ = showTreeHorizontally tr;;

let tr = remove 3 tr;;
let test_remove1 = tr = Node (6, 3, Node (4, 2, Empty, Node (5, 1, Empty, Empty)),
   Node (7, 1, Empty, Empty))
let _ = showTreeHorizontally tr;;

let tr = remove 4 tr;;
let test_remove1 = tr = Node (6, 2, Node (5, 1, Empty, Empty), Node (7, 1, Empty, Empty))
let _ = showTreeHorizontally tr;;

let tr = remove 5 tr;;
let test_remove1 = tr = Node (6, 2, Empty, Node (7, 1, Empty, Empty))
let _ = showTreeHorizontally tr;;

let tr = remove 6 tr;;
let test_remove1 = tr = Node (7, 1, Empty, Empty)
let _ = showTreeHorizontally tr;;

let tr = remove 7 tr;;
let test_remove1 = tr = Empty
let _ = showTreeHorizontally tr;;

[Vir](https://www.cs.cornell.edu/courses/cs312/2007fa/lectures/lec14.html).